# Import Modules

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime

# Classification using Neural Networks

In [22]:
stemmer = LancasterStemmer()
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# 2 classes of training data
training_data = []
training_data.append({"class":"hotel", "sentence":"We stayed at this hotel for 3 nights"})
training_data.append({"class":"hotel", "sentence":"This hotel is in a great location"})
training_data.append({"class":"hotel", "sentence":"Chambre propre et agréable"})
training_data.append({"class":"hotel", "sentence":"We really enjoyed staying"})
training_data.append({"class":"hotel", "sentence":"I couldn't say enough great things about this hotel"})
training_data.append({"class":"hotel", "sentence":"El b&b está bien situado en plena vía cavour"})
training_data.append({"class":"hotel", "sentence":"Località al centro di Roma"})
training_data.append({"class":"hotel", "sentence":"They said not a problem and greeted me at arrival"})
training_data.append({"class":"hotel", "sentence":"Great location"})
training_data.append({"class":"hotel", "sentence":"Great stay on a central location"})
training_data.append({"class":"hotel", "sentence":"agréable séjour"})
training_data.append({"class":"hotel", "sentence":"Excellent hotel"})
training_data.append({"class":"hotel", "sentence":"Hôtel propre"})

training_data.append({"class":"restaurent", "sentence":"Excellent restaurant"})
training_data.append({"class":"restaurent", "sentence":"une cuisine raffinée"})
training_data.append({"class":"restaurent", "sentence":"Tout ce que l’on cherche pour passer un agréable moment c’est passé dans ce restaurant"})
training_data.append({"class":"restaurent", "sentence":"Plats délicieux et bien cuisinés"})
training_data.append({"class":"restaurent", "sentence":"la nourriture est bonne"})
training_data.append({"class":"restaurent", "sentence":"Ambiance branchée pour une très bonne restauration"})
training_data.append({"class":"restaurent", "sentence":"Good quality food"})
training_data.append({"class":"restaurent", "sentence":"The food was delicious"})
training_data.append({"class":"restaurent", "sentence":"Very elegant and luxury restaurant"})
training_data.append({"class":"restaurent", "sentence":"very good cocktails, food and wine don't miss this"})

print ("%s sentences in training data" % len(training_data))
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    w = nltk.word_tokenize(pattern['sentence'])
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remove duplicates
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

# sample training/output
i = 0
w = documents[i][0]
print ([stemmer.stem(word.lower()) for word in w])
print (training[i])
print (output[i])
import numpy as np
import time

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # how much did we miss the target value?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20, alpha=0.1, epochs=100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")
# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results


[nltk_data] Downloading package punkt to /Users/alaazzabi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
23 sentences in training data
23 documents
2 classes ['hotel', 'restaurent']
94 unique stemmed words ['real', 'plen', 'win', 'hotel', 'di', 'passé', 'propr', 'chambr', 'enjoy', 'cent', '&', 'raffiné', '’', 'al', 'delicy', "n't", 'branché', 'est', 'località', 'good', 'séjour', 'amby', 'do', 'la', 'me', 'hôtel', 'thi', 'is', 'cuisiné', 'plat', 'qual', 'in', 'a', 'miss', '3', 'délicieux', 'cherch', 'que', 'agré', ',', 'resta', 'was', 'bien', 'greet', 'luxury', 'cuisin', 'très', 'eleg', 'on', 'situado', 'stay', 'en', 'the', 'c', 'ar', 'tout', 'and', 'they', 'el', 'gre', 'et', 'b', 'for', 'nourrit', 'vía', 'we', 'i', 'say', 'dan', 'about', 'problem', 'un', 'pass', 'very', 'food', 'could', 'cocktail', 'rom', 'enough', 'cavo', 'l', 'pour', 'loc', 'bon', 'centro', 'said', 'está', 'thing', 'mom', 'at', 'ce', 'not', 'night', 'excel']
['we', 'stay', 'at', 'thi', 'hotel', 'for'

# Test & Results

In [23]:
classify("Wonderful stay - everything perfect")
classify("Good location, nice room")
classify("Nous avons séjourné en famille")
classify("I had a reservation at this hotel for myself and 2 friends")
classify("Nous avons suivi les recommandations de commandes sur les plats")
classify("Always excellent Food and service Preference goes to Ceviche")
print()
classify("Spent 5 nights at this little gem", show_details=True)

Wonderful stay - everything perfect 
 classification: [['hotel', 0.98530052479345387]]
Good location, nice room 
 classification: [['restaurent', 0.55213662055917745], ['hotel', 0.390372899064935]]
Nous avons séjourné en famille 
 classification: [['hotel', 0.67888200131853516], ['restaurent', 0.32511921997444726]]
I had a reservation at this hotel for myself and 2 friends 
 classification: [['hotel', 0.99964901303042053]]
Nous avons suivi les recommandations de commandes sur les plats 
 classification: [['restaurent', 0.60113834723433346], ['hotel', 0.52544511363797519]]
Always excellent Food and service Preference goes to Ceviche 
 classification: [['restaurent', 0.98007030455069233]]

found in bag: night
found in bag: at
found in bag: thi
sentence: Spent 5 nights at this little gem 
 bow: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0]
Spent

[['hotel', 0.99353594923442279]]